## 4-4. 한국어 텍스트의 카운트 벡터 변환
한국어 텍스트로부터 카운트 벡터를 생성하는 단계는 기본적으로 영어와 동일한데, 차이가 있다면 형태소 분석기뿐이라고 할 수 있다. 한글에 대한 분석을 할 때에는 CountVectorizer가 제공하는 기본 토크나이저를 쓸 수 없다. 따라서 KoNLPy의 형태소 분석기를 지정하여 CountVectorizer 객체를 선언해야 한다. 다음은 인터넷에 공개된 한국어 데이터셋들로, 네이버 영화 리뷰는 평점이 있어 감성분석이 가능하며, 뉴스 뎅터는 카테고리 분류용으로 사용할 수 있다.

- [네이버 영화 리뷰, 텍스트 파일로 제공](https://github.com/e9t/nsmc)
- [네이버 뉴스 분류, NNST 파이썬 라이브러리 형태로 제공](https://github.com/jason9693/NNST-Naver-News-for-Standard-and-Technology-Database)

이 책에서는 Daum 영화 리뷰를 크롤링해 만든 데이터를 사용한다. 데이터는 리뷰 내용, 평점, 날짜, 영화 제목으로 이루어져 있다. 이 데이터에는 평점과 영화 제목이 다 있으므로 감성분석과 영화 제목 예측 혹은 분류가 모두 가능하다. 또, 날짜가 있어 나중에 토픽 트렌드를 살펴볼 수도 있다. 그러나 이 절에서는 단지 리뷰 내용을 어떻게 카운트 벡터로 변환할지에 초점을 맞추므로 리뷰 내용에만 집중한다.

### 데이터 살펴보기

In [1]:
import pandas as pd

df = pd.read_csv('../data/daum_movie_review.csv')

df.head(10)

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워
5,나는 재밌게 봄,10,2018.10.14,인피니티 워
6,0.5점은 줄 수 없냐?,0,2018.10.10,인피니티 워
7,헐..다 죽었어....나중에 앤트맨 보다가도 깜놀...,10,2018.10.08,인피니티 워
8,충격 결말,9,2018.10.06,인피니티 워
9,응집력,8,2018.10.05,인피니티 워


위 예를 보면 영화리뷰 데이터가 분석하기 쉽지 않다는 것을 알 수 있다. 내용이 상세한 경우에는 그나마 괜찮지만, 어떤 리뷰는 너무 짧아서 과연 이걸로 감성분석이나 제목 예측이 가능할까 싶다. 위 리뷰 중에서 4번이나 5번의 내용을 보고 '인피니티 워'라는 제목을 맞출 수 있을까? 우선 여기서는 review 항목으로부터 카운트 벡터를 생성하는 것에 집중한다. 먼저 영어 문서분류와 동일하게 자체 토크나이저를 사용해서 아래와 같이 카운트 벡터를 생성한다.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

daum_cv = CountVectorizer(max_features=1000)

daum_DTM = daum_cv.fit_transform(df.review) # review를 이용해 count vector를 학습하고 변환

print(daum_cv.get_feature_names_out()[:100]) # count vector에 사용된 feature 이름을 반환

['10점' '18' '1987' '1도' '1점' '1점도' '2시간' '2시간이' '2편' '5점' '6점' '7점' '8점'
 'cg' 'cg가' 'cg는' 'cg도' 'cg만' 'good' 'of' 'ㅋㅋ' 'ㅋㅋㅋ' 'ㅋㅋㅋㅋ' 'ㅎㅎ' 'ㅎㅎㅎ'
 'ㅜㅜ' 'ㅠㅠ' 'ㅠㅠㅠ' 'ㅡㅡ' '가는' '가는줄' '가면' '가서' '가슴' '가슴아픈' '가슴이' '가장' '가족'
 '가족과' '가족들과' '가족의' '가족이' '가지고' '간만에' '갈수록' '감독' '감독님' '감독은' '감독의' '감독이'
 '감동' '감동과' '감동도' '감동은' '감동을' '감동이' '감동입니다' '감동적' '감동적이고' '감동적인' '감사드립니다'
 '감사합니다' '감정이' '갑자기' '갔는데' '갔다가' '강철비' '강추' '강추합니다' '같고' '같네요' '같다' '같습니다'
 '같아' '같아요' '같은' '같은데' '같음' '같이' '개연성' '개연성이' '개인적으로' '거의' '겁나' '것도' '것은'
 '것을' '것이' '것이다' '겨울왕국' '결국' '결말' '결말이' '계속' '고맙습니다' '곤지암' '공포' '공포를'
 '공포영화' '관객']


위 결과는 문제가 많아 보이는데, 'cg', '감동' 등이 들어간 단어들이 전부 별도의 단어로 분류됐다. 이런 단어들은 의미적으로 봤을 때 같은 단어로 취급하는 것이 바람직한데, 영어룰 대상으로 하는 기본 토크나이저는 적합하지 않았음을 알 수 있다. 이에 KoNLPy의 형태소 분석기를 사용해야 하고, 아래와 같이 형태소 분석기를 불러와 간단히 테스트해본다.

In [3]:
from konlpy.tag import Okt

twitter_tag = Okt()

print('# 전체 형태소 결과:', twitter_tag.morphs(df.review[1]))
print('# 명사만 추출:', twitter_tag.nouns(df.review[1]))
print('# 품사 태깅 결과:', twitter_tag.pos(df.review[1]))

# 전체 형태소 결과: ['몰입', '할수밖에', '없다', '.', '어렵게', '생각', '할', '필요없다', '.', '내', '가', '전투', '에', '참여', '한', '듯', '손', '에', '땀', '이남', '.']
# 명사만 추출: ['몰입', '생각', '내', '전투', '참여', '듯', '손', '땀', '이남']
# 품사 태깅 결과: [('몰입', 'Noun'), ('할수밖에', 'Verb'), ('없다', 'Adjective'), ('.', 'Punctuation'), ('어렵게', 'Adjective'), ('생각', 'Noun'), ('할', 'Verb'), ('필요없다', 'Adjective'), ('.', 'Punctuation'), ('내', 'Noun'), ('가', 'Josa'), ('전투', 'Noun'), ('에', 'Josa'), ('참여', 'Noun'), ('한', 'Determiner'), ('듯', 'Noun'), ('손', 'Noun'), ('에', 'Josa'), ('땀', 'Noun'), ('이남', 'Noun'), ('.', 'Punctuation')]


위 결과를 보면 전체 형태소를 다 사용하는 경우에는 필요 없는 단어가 많아 보이고, 그렇다고 명사만 쓰면 또 너무 적어 보인다. 품사 태깅 결과를 보면 Josa나 Punctuation은 빼는 것이 좋아 보인다. 어디까지 특성 집합에 포함할지는 전적으로 실행하는 사람의 선택이다. 여기서는 명사, 동사, 형용사 세 개를 선택하기로 한다. CountVectorizer는 토크나이저를 함수 형태로 지정할 수 있으므로, 나만의 토크나이저 함수를 아래와 같이 만들고 테스트한다.

In [4]:
def my_tokenizer(doc):
    return [
        token
        for token, pos in twitter_tag.pos(doc)
        if pos in ['Noun', 'Verb', 'Adjective']
    ]

print('나만의 토크나이저 결과:', my_tokenizer(df.review[1]))

나만의 토크나이저 결과: ['몰입', '할수밖에', '없다', '어렵게', '생각', '할', '필요없다', '내', '전투', '참여', '듯', '손', '땀', '이남']


일단 이 정도로 만족하기로 하고, 아래와 같이 tokenizer 매개변수에 my_tokenizer 함수를 지정해 객체를 생성한 후 fit_transform으로 카운트 벡터를 생성한다. CountVectorizer에서 토크나이저로 매번 my_tokenizer를 호출하므로 영어와는 달리 시간이 꽤 오래 걸릴 수 있다.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# 토크나이저와 특성의 최대 개수를 지정
# 명사만 추출하고 싶은 경우에는 tokenizer에 'twitter_tag.nouns'를 바로 지정해도 됨
daum_cv = CountVectorizer(max_features=1000, tokenizer=my_tokenizer)

# review를 이용해 count vector를 학습하고 변환
daum_DTM = daum_cv.fit_transform(df.review)

print(daum_cv.get_feature_names_out()[:100]) # count vector에 사용된 feature 이름을 반환

['가' '가는' '가는줄' '가면' '가서' '가슴' '가장' '가족' '가족영화' '가지' '가치' '각색' '간' '간다'
 '간만' '갈' '갈수록' '감' '감독' '감동' '감사' '감사합니다' '감상' '감성' '감정' '감탄' '갑자기' '갔는데'
 '갔다' '갔다가' '강' '강철' '강추' '같고' '같네요' '같다' '같습니다' '같아' '같아요' '같은' '같은데'
 '같음' '개' '개그' '개봉' '개연' '개인' '거' '거기' '거리' '거의' '걱정' '건' '건가' '건지' '걸'
 '겁니다' '것' '게' '겨울왕국' '결론' '결말' '경찰' '경험' '계속' '고' '고맙습니다' '고민' '고생' '곤지암'
 '곳' '공감' '공포' '공포영화' '과' '과거' '관' '관객' '관객수' '관람' '광주' '괜찮은' '교훈' '구성'
 '국내' '국민' '군인' '군함도' '굿' '권선' '귀신' '그' '그것' '그게' '그날' '그냥' '그닥' '그대로'
 '그때' '그래픽']


위에서 있었던 '감독', '감동' 등의 문제는 해결된 것으로 보인다. daum_BOW의 정보를 살짝 들여다보고, 얼마나 희소한지 계산해보자.

In [6]:
print(repr(daum_DTM))

<14725x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 110800 stored elements in Compressed Sparse Row format>


In [7]:
print(110800/(14725*1000))

0.007524617996604414


희소행렬에서 값이 있는 비율이 0.75%밖에 안 된다. 이는 위에서 살펴본 NLTK의 영화 리뷰에 비해 훨씬 적은 값인데, NLTK의 리뷰는 비교가 길고 사용된 단어가 많은 반면, Daum 영화 리뷰는 짧은 것이 대부분이기 때문이다. 위에서 예제로 본 둘째 리뷰에 대해 사용된 단어와 개수를 보고 싶으면 아래와 같이 할 수 있다.

In [8]:
for word, count in zip(daum_cv.get_feature_names_out(), daum_DTM[1].toarray()[0]):
    if count > 0:
        print(word, ':', count, end=', ')

내 : 1, 듯 : 1, 몰입 : 1, 생각 : 1, 손 : 1, 없다 : 1, 할 : 1, 

위의 토큰화 결과에 비해 개수가 줄었는데, 우리가 max_feature를 1,000으로 제한해서 빈도가 낮은 단어들은 제외했기 때문이다. 이상에서 살펴본 바와 같이 한글에 대해 카운터 벡터 생성은 한글 형태소 분석기를 토크나이저로 지정해주는 것 외에 영어와 큰 차이는 없다.